### 스포츠 센터의 탈퇴를 지도학습의 분류를 이용해서 예측

- 앞장에서 소개한 클러스터링을 통한 행동 분석은 사용 방법에 따라 많은 가능성이 있는 기술입니다. 행동 패턴을 분석할 수 있으면 어떤 고객이 탈퇴할지와 같은 예측도 어느 정도 정확하게 할 수 있기 때문에 탈퇴를 방지하기 위한 정책을 미리 준비하는 것도 가능합니다.

- 이 장에서는 이미 탈퇴한 회원과 계속해서 이용하는 회원의 데이터를 가지고 '의사결정 트리(Decision Tree)'라고 부르는 지도학습의 분류 알고리즘을 이용해서 탈퇴를 예측하는 흐름을 배웁니다.           
의사결정 트리는 간단한 수법이지만 알기 쉽게 원인 분석을 할 수 있기 때문에 현장에서 자주 사용됩니다.

고객의 소리) 상세 분석을 고려하여 생각해 보면 회원을 정착시키고 늘려가는 것보다 탈퇴를 막는 것이 중요한 것 같습니다. <u>탈퇴 회원이 왜 탈퇴했는지 알 수 있을까요?</u>

데이터셋)
1. use_log.csv : 센터 이용 이력(회원이 센터를 이용하면 시스템에 자동 입력 - 2018년 4월 ~ 2019년 3월의 1년분 데이터)     
2. customer_master.csv : 2019년 3월 말 시점의 회원 데이터(이전에 탈퇴한 회원 포함)             
3. class_master.csv : 회원 구분 데이터(종일, 주간, 야간)           
4. campaign_master.csv : 가입 시 행사 종류 데이터(입회비 유무 등)       
5. customer_use_merge_df.csv : 이전에 작성한 이용 이력을 포함한 고객 데이터               
6. monthly_use_df.csv : 이용 이력을 '연월/회원별'로 집계한 데이터 <b><u>New</u></b>


In [1]:
import pandas as pd 

customer_merge_use_df = pd.read_csv('./data/customer_use_merge_df.csv')
customer_merge_use_df.head()

monthly_use_df = pd.read_csv('./data/monthly_use_df.csv')
monthly_use_df.head()
# monthly_use_df.info()


,customer_id,use_month,count
0,AS002855,201804,4
1,AS002855,201805,5
2,AS002855,201806,5
3,AS002855,201807,5
4,AS002855,201808,3


In [2]:
customer_merge_use_df.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,regularity,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,0_종일,10500,2_일반,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,0_종일,10500,2_일반,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,0_종일,10500,2_일반,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,0_종일,10500,2_일반,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,2_야간,6000,2_일반,3.916667,4.0,6,1,1,2019-04-30,47


미래를 예측하기 위해 그 달과 1개월 전의 이용 이력만으로 데이터를 작성합니다.        
4장에서와 같이 과거 6개월분의 데이터로부터 이용 횟수를 예측하는 경우, 가입 5개월 이내인 회원의 탈퇴는 예측할 수 없습니다.   
불과 몇 개월 만에 그만둔 회원도 많기 때문에 과거 6개월분의 데이터를 이용한 예측은 의미가 없습니다.

In [3]:
monthly_use_df['use_month'] == 201804
monthly_use_df['use_month'].dtype

# monthly_use_df.loc[monthly_use_df['use_month'] == 201804] # copying 문제로 loc을 쓰는 버릇** 필요

dtype('int64')

In [4]:
total_year_months = monthly_use_df['use_month'].unique()
total_year_months

data_for_ml = pd.DataFrame()
for i in range(1, len(total_year_months)) :
  tmp_pred = monthly_use_df.loc[monthly_use_df['use_month'] == total_year_months[i]]
  tmp_pred.rename(columns= {'count' : 'count_pred'}, inplace= True)
  tmp_before = monthly_use_df.loc[monthly_use_df['use_month'] == total_year_months[i - 1]]
  tmp_before.drop('use_month', axis= 1, inplace= True)
  tmp_before.rename(columns= {'count' : 'count_before'}, inplace= True)
  tmp = tmp_pred.merge(tmp_before, on= 'customer_id', how= 'left')
  data_for_ml = pd.concat([data_for_ml, tmp], axis= 0, ignore_index= True)

data_for_ml.head()

C:\Users\LGCARE\AppData\Local\Temp\ipykernel_14412\875998746.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_pred.rename(columns= {'count' : 'count_pred'}, inplace= True)
C:\Users\LGCARE\AppData\Local\Temp\ipykernel_14412\875998746.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_before.drop('use_month', axis= 1, inplace= True)
C:\Users\LGCARE\AppData\Local\Temp\ipykernel_14412\875998746.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

,customer_id,use_month,count_pred,count_before
0,AS002855,201805,5,4.0
1,AS009373,201805,4,3.0
2,AS015233,201805,7,NaN
3,AS015315,201805,3,6.0
4,AS015739,201805,5,7.0


>> 탈퇴 회원의 데이터 작성(탈퇴 전월)

탈퇴 회원의 회원 정보와 사용 기록을 결합         
이 때, 탈퇴 신청 월을 기준으로 사용 기록을 결합(직전 월의 데이터가 예측에 중요)

2018년 8월  |  2018년 9월    |  2018년 10월              
탈퇴 신청    | 탈퇴 신청 완료  |      탈퇴                 
탈퇴 전월    | 탈퇴 월

- 왜 end_date 칼럼의 탈퇴 월이 아닌 탈퇴 전월의 데이터를 작성할까요?              
- 탈퇴를 예측하는 목적은 미연에 방지하는 것에 있습니다. 이 스포츠 센터에서는 월말까지 탈퇴 신청을 해야 다음 달 말에 탈퇴할 수 있습니다. 예를 들어, 2018년 9월 30일에 탈퇴(end_date)한 회원은 8월에 탈퇴 신청을 했기 때문에 9월의 데이터를 사용하는 것은 탈퇴를 방지할 수 없습니다. - <b><u>8월의 데이터(탈퇴 신청 달)를 사용해서 9월의 탈퇴(실제 탈퇴 달)를 예측해야 함</u></b>

- 탈퇴한 회원 추출
- end_date의 1개월 전을 계산하여 저장
- data_fol_ml(현재 + 직전월 사용 기록)와 customer_id, use_month를 키로 결합

In [5]:
quit_customers_df = customer_merge_use_df.loc[customer_merge_use_df['is_deleted'] == 1]
quit_customers_df.reset_index(inplace= True, drop= True)
quit_customers_df.head()

from dateutil.relativedelta import relativedelta

quit_customers_df['quit_signup'] = None # 탈퇴 신청 월
quit_customers_df['end_date'] = pd.to_datetime(quit_customers_df['end_date'])

for i in range(len(quit_customers_df)) :
  quit_customers_df['quit_signup'].iloc[i] = quit_customers_df['end_date'].iloc[i] - relativedelta(months= 1) # 시각까지 포함된 값

quit_customers_df.head()

quit_customers_df['quit_signup'] = pd.to_datetime(quit_customers_df['quit_signup']).dt.strftime('%Y%m')
data_for_ml['use_month'] = data_for_ml['use_month'].astype(str)

# 'left' : 왼쪽의 모든 행을 유지하면서 오른쪽에서 on 조건이 맞으면 붙이고 없으면 NaN을 채운다는 뜻
# data_for_ml : customer_id | use_month | count_pred | count_before
# quit_cutomser_df : 탈퇴 회원만
quit_customer_monthly_use_df = data_for_ml.merge(quit_customers_df, how= 'left', 
                                                 left_on= ['customer_id', 'use_month'], right_on = ['customer_id', 'quit_signup'])
# > 탈퇴 신청한 월의 사용 횟수

print(len(monthly_use_df)) # 36842 - 사용 로그 기준
print(len(quit_customer_monthly_use_df)) # 33851
quit_customer_monthly_use_df.head()

# javascript로도 돌아가네?


C:\Users\LGCARE\AppData\Local\Temp\ipykernel_14412\3233479928.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quit_customers_df['quit_signup'] = None # 탈퇴 신청 월
C:\Users\LGCARE\AppData\Local\Temp\ipykernel_14412\3233479928.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quit_customers_df['end_date'] = pd.to_datetime(quit_customers_df['end_date'])
C:\Users\LGCARE\AppData\Local\Temp\ipykernel_14412\3233479928.py:11: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are set

36842
33851


,customer_id,use_month,count_pred,count_before,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,regularity,calc_date,membership_period,quit_signup
0,AS002855,201805,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AS009373,201805,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AS015233,201805,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AS015315,201805,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AS015739,201805,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


결합한 데이터는 탈퇴한 회원의 탈퇴 전월의 데이터뿐이므로 결측치가 많습니다.                 
use_month는 있는데 withdrawl_application이 nan인 경우 - <b><u>사용 기록이 있는 회원이 그 달에 탈퇴하지 않은 것</u></b>         
사용 기록이 있는 경우는 전부 불러오는 merge

- quit_use_log : 탈퇴한 회원의 탈퇴 신청한 달의 사용 기록 & 고객 정보

In [6]:
quit_use_log = quit_customer_monthly_use_df.dropna(subset= 'name') # name이 nan이면 탈퇴하지 않은 것(name부터의 열은 quit_signup 데이터프레임)
print(len(quit_use_log))
print(len(quit_use_log['customer_id'].unique()))
quit_use_log.head() # 탈퇴한 회원의 탈퇴 신청한 달의 사용 기록 & 고객 정보
# 어떤 특정 회원이 그만두기 전월의 상태를 나타내는 데이터


1104
1104


,customer_id,use_month,count_pred,count_before,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,regularity,calc_date,membership_period,quit_signup
19,AS055680,201805,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,2_일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,201805
57,AS169823,201805,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,2_일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,201805
110,AS305860,201805,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,201805
128,AS363699,201805,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,201805
147,AS417696,201805,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,2_일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,201805


>> 지속 회원의 데이터 작성

지속 회원은 탈퇴 월이 없기 때문에 어떤 연월의 데이터를 작성해도 됩니다. 따라서 지속 회원을 추출한 후 uselog 데이터에 결합하면 됩니다.

In [7]:
continue_customer_df = customer_merge_use_df.loc[customer_merge_use_df['is_deleted'] == 0]
continue_customer_df.head()

continue_use_log = pd.merge(data_for_ml, continue_customer_df, on= 'customer_id', how= 'left')
print(len(continue_use_log)) # 33851
continue_use_log = continue_use_log.dropna(subset=['name'])
print(len(continue_use_log))

# data_for_ml : 월별 사용 기록 + 직전 월 사용 기록
# data_for_ml에는 있으나 continue_customer_df에는 없는 경우 > 탈퇴한 회원에 대한 데이터

# data_for_ml에 하나의 customer_id에 use_month가 2018년 4월, 5월인 경우 고객 데이터가 각각 붙음
# 탈퇴 월이 없기 때문에 2018년 5월 A씨의 데이터나 2018년 12월 A씨의 데이터, 아무거나 지속 고객의 데이터로 사용 가능

33851
27422


In [8]:
# 데이터 불균형 지속회원 : 29576 vs 탈퇴 회원 : 1104
# 지속 회원 > 회원당 1개로 언더샘플링
# 2018년 5월 A씨와 2018년 12월 A씨 중 하나만 선택

# 데이터를 섞고 중복을 제거 - 처음 나온 데이터를 가져옴
continue_use_log = continue_use_log.sample(frac= 1).reset_index(drop= True)
continue_use_log = continue_use_log.drop_duplicates(subset= 'customer_id')
print(len(continue_use_log)) # 2842
continue_use_log.head() 

2842


,customer_id,use_month,count_pred,count_before,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,regularity,calc_date,membership_period
0,TS023216,201807,5,1.0,XXXXX,C02,F,2015-07-01,NaN,CA1,...,1_주간,7500.0,2_일반,4.250000,4.0,8.0,1.0,1.0,2019-04-30,45.0
1,AS436928,201809,5,5.0,XXX,C01,F,2016-07-01,NaN,CA2,...,0_종일,10500.0,0_입회비반액할인,4.583333,5.0,6.0,2.0,1.0,2019-04-30,33.0
2,TS895361,201809,6,6.0,XXXXX,C02,F,2017-07-01,NaN,CA2,...,1_주간,7500.0,0_입회비반액할인,5.500000,6.0,8.0,3.0,1.0,2019-04-30,21.0
3,HI109765,201805,7,4.0,XXXXX,C01,M,2017-01-01,NaN,CA1,...,0_종일,10500.0,2_일반,4.833333,5.0,7.0,3.0,1.0,2019-04-30,27.0
4,PL983142,201902,11,7.0,XXXX,C02,F,2019-01-07,NaN,CA1,...,1_주간,7500.0,2_일반,8.333333,7.0,11.0,7.0,1.0,2019-04-30,3.0


>> 지속 회원 데이터, 탈퇴 회원 데이터 결합

In [9]:
predict_data = pd.concat([quit_use_log, continue_use_log], axis= 0, ignore_index= True)
print(len(predict_data)) # 3946
predict_data.head()

3946


C:\Users\LGCARE\AppData\Local\Temp\ipykernel_14412\3923030628.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predict_data = pd.concat([quit_use_log, continue_use_log], axis= 0, ignore_index= True)


,customer_id,use_month,count_pred,count_before,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,regularity,calc_date,membership_period,quit_signup
0,AS055680,201805,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,2_일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,201805
1,AS169823,201805,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,2_일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,201805
2,AS305860,201805,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,201805
3,AS363699,201805,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,201805
4,AS417696,201805,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,2_일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,201805


>> 예측할 달의 재적 기간 작성

시간적 요소가 들어간 데이터이므로 재적 기간과 같은 데이터를 변수로 이용하는 것이 좋은 접근이라고 할 수 있습니다.

In [10]:
predict_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3946 entries, 0 to 3945
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   customer_id        3946 non-null   object        
 1   use_month          3946 non-null   object        
 2   count_pred         3946 non-null   int64         
 3   count_before       3676 non-null   float64       
 4   name               3946 non-null   object        
 5   class              3946 non-null   object        
 6   gender             3946 non-null   object        
 7   start_date         3946 non-null   object        
 8   end_date           1104 non-null   datetime64[ns]
 9   campaign_id        3946 non-null   object        
 10  is_deleted         3946 non-null   float64       
 11  class_name         3946 non-null   object        
 12  price              3946 non-null   float64       
 13  campaign_name      3946 non-null   object        
 14  mean    

In [11]:
# count_pred 예측할 달(use_month) 기준으로 재적 기간 계산
# count_before은 탈퇴 신청 월의 사용 횟수
predict_data['now_date'] = pd.to_datetime(predict_data['use_month'], format= '%Y%m')
predict_data['start_date'] = pd.to_datetime(predict_data['start_date'])

predict_data['period'] = 0 # None으로 설정할 경우 dtype > object
for i in range(len(predict_data)):
  delta = relativedelta(predict_data.loc[i, 'now_date'],predict_data.loc[i, 'start_date'])
  predict_data.loc[i, 'period'] = int(delta.years * 12 + delta.months)

predict_data.head()

,customer_id,use_month,count_pred,count_before,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,regularity,calc_date,membership_period,quit_signup,now_date,period
0,AS055680,201805,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,201805,2018-05-01,2
1,AS169823,201805,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,201805,2018-05-01,6
2,AS305860,201805,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,201805,2018-05-01,11
3,AS363699,201805,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,201805,2018-05-01,3
4,AS417696,201805,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,201805,2018-05-01,8


>> 머신러닝 준비

>> 결측치 처리

In [12]:
# predict_data.info()
predict_data.isnull().sum() 

# > end_date, quit_signup 은 탈퇴 고객만 존재하므로 결측치 존재
# > count_before의 경우 전월 이용 기록이 없는 데이터 > 삭제

customer_id             0
use_month               0
count_pred              0
count_before          270
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
regularity              0
calc_date               0
membership_period       0
quit_signup          2842
now_date                0
period                  0
dtype: int64

In [13]:
drop_predict_data = predict_data.dropna(axis= 0, how= 'any', subset= 'count_before', ignore_index= True)
drop_predict_data.isnull().sum() # 2639

customer_id             0
use_month               0
count_pred              0
count_before            0
name                    0
class                   0
gender                  0
start_date              0
end_date             2624
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
regularity              0
calc_date               0
membership_period       0
quit_signup          2624
now_date                0
period                  0
dtype: int64

>> 문자열 변수 처리

가입 캠페인 구분, 회원 구분, 성별과 같은 문자열 데이터는 어떻게 처리하면 좋을까요?         
이런 데이터를 카테고리 변수라고 합니다. 이런 데이터도 머신러닝을 하는 데 있어 중요한 변수가 됩니다. 이런 데이터를 활용하기 위해 앞에서 routine_flg(regularity)를 작성한 것처럼 플래그를 만듭니다. 이것을 "더미 변수"라고 합니다.

설명 변수 : count_before, (카테고리 변수)campaign_name, class_name, gender, regularity, 재적 기간 period를 사용            
목적 변수 : is_deleted
       
** 지도 학습 - 분류

In [14]:
classify_df = drop_predict_data[['count_before', 'campaign_name', 'class_name', 'gender', 'regularity', 'period', 'is_deleted']]
classify_df.info()
classify_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3676 entries, 0 to 3675
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   count_before   3676 non-null   float64
 1   campaign_name  3676 non-null   object 
 2   class_name     3676 non-null   object 
 3   gender         3676 non-null   object 
 4   regularity     3676 non-null   float64
 5   period         3676 non-null   int64  
 6   is_deleted     3676 non-null   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 201.2+ KB


,count_before,campaign_name,class_name,gender,regularity,period,is_deleted
0,3.0,2_일반,0_종일,M,0.0,2,1.0
1,3.0,2_일반,0_종일,M,1.0,6,1.0
2,3.0,2_일반,0_종일,M,0.0,11,1.0
3,3.0,2_일반,0_종일,M,0.0,3,1.0
4,4.0,2_일반,2_야간,F,0.0,8,1.0


>> 카테고리 변수 처리

In [15]:
# classify_df = pd.get_dummies(classify_df, columns= ['campaign_name', 'class_name', 'gender'])
classify_df = pd.get_dummies(classify_df)
classify_df.head()

,count_before,regularity,period,is_deleted,campaign_name_0_입회비반액할인,campaign_name_1_입회비무료,campaign_name_2_일반,class_name_0_종일,class_name_1_주간,class_name_2_야간,gender_F,gender_M
0,3.0,0.0,2,1.0,False,False,True,True,False,False,False,True
1,3.0,1.0,6,1.0,False,False,True,True,False,False,False,True
2,3.0,0.0,11,1.0,False,False,True,True,False,False,False,True
3,3.0,0.0,3,1.0,False,False,True,True,False,False,False,True
4,4.0,0.0,8,1.0,False,False,True,False,False,True,True,False


- gender_F, gender_M의 경우 한 변수로 다른 변수 설명 가능               
- campaign_name_0_입회비무료, campaign_name_1_입회비무료, campaign_name_2_일반의 경우 2개의 변수로 다른 1개의 변수 설명 가능       
- class_name_0_종일, class_name_1_주간, class_name_2_야간의 경우 2개의 변수로 다른 1개의 변수 설명 가능     


In [16]:
classify_df = classify_df.drop(['campaign_name_2_일반', 'class_name_2_야간', 'gender_M'], axis= 1)
classify_df.head()

,count_before,regularity,period,is_deleted,campaign_name_0_입회비반액할인,campaign_name_1_입회비무료,class_name_0_종일,class_name_1_주간,gender_F
0,3.0,0.0,2,1.0,False,False,True,False,False
1,3.0,1.0,6,1.0,False,False,True,False,False
2,3.0,0.0,11,1.0,False,False,True,False,False
3,3.0,0.0,3,1.0,False,False,True,False,False
4,4.0,0.0,8,1.0,False,False,False,False,True


>> 의사결정 트리를 사용한 탈퇴 예측 모델 구축

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

quit_df = classify_df.loc[classify_df['is_deleted'] == 1] # 1104
continue_df = classify_df.loc[classify_df['is_deleted'] == 0].sample(len(quit_df)) # 2842

X = pd.concat([quit_df, continue_df], axis= 0, ignore_index= True)
# X.head()
y = X['is_deleted']
X.drop('is_deleted', axis= 1, inplace= True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

tree_clf = DecisionTreeClassifier(random_state= 0)
tree_clf.fit(X_train, y_train)
y_preds = tree_clf.predict(X_test)
y_preds # 1이 탈퇴 0이 유지


array([0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1.,
       1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0.,
       1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1.,
       0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0.,
       1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
       1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0.,
       1., 1., 0., 0., 1.

In [18]:
# 실제 정답과 비교

test_res = pd.DataFrame({'y_test' : y_test, 'y_preds' : y_preds})
test_res.head()

,y_test,y_preds
1924,0.0,0.0
1284,0.0,0.0
624,1.0,1.0
42,1.0,0.0
1507,0.0,0.0


>> 모델 평가 및 튜닝

In [19]:
# 정답률 집계

correct = len(test_res.loc[test_res['y_test'] == test_res['y_preds']]) # 474
data_count = len(test_res)

score_test = correct / data_count
print(score_test) # 90.11%


0.8916349809885932


머신러닝의 목적은 미지의 데이터를 이용한 예측으로 <b><u>학습용 데이터로 예측한 정확도</u></b>와 <u><b>평가용 데이터로 예측한 정확도의 차이가 작</u></b>은 것이 이상적입니다.

In [20]:
train_score = tree_clf.score(X_train, y_train)
test_score = tree_clf.score(X_test, y_test)
print('train score :', train_score)
print('test score:', test_score)

# 정확도)
# train : 98.1%
# test : 90.1%

train score : 0.9797211660329531
test score: 0.8916349809885932


학습용 데이터에 대한 과적합 현상
- 데이터 늘리기
- 변수 재검토
- 모델의 파라미터 변경

의사결정 트리는 단적으로 말하면 가장 깨끗하게 0과 1로 분할할 수 있는 설명 변수와 그 조건에 맞는 트리 구조를 찾는 방법입니다.        
따라서 <u>트리의 깊이를 얕게 하면</u> 모델을 단순화할 수 있습니다.

In [21]:
simplified_tree_clf = DecisionTreeClassifier(max_depth= 5, random_state= 0)
simplified_tree_clf.fit(X_train, y_train)

y_preds = simplified_tree_clf.predict(X_test)

In [22]:
simplified_tree_clf.score(X_train, y_train) # 93.16%
simplified_tree_clf.score(X_test, y_test) # 91.1%

0.9372623574144486

max_depth를 5로 설정한 모델의 스코어를 보면 <b><u>학습용 데이터로 예측한 정확도</u></b>와 <u><b>평가용 데이터로 예측한 정확도의 차이가 작</u></b>아졌습니다.           
max_depth를 지정하지 않은 쪽이 학습용 데이터의 정답률이 높지만 과적합에 의해 평가용 데이터의 정답률이 낮아집니다.         

>> 모델에 기여하는 변수 확인

In [24]:
feature_importance = pd.DataFrame({'feature' : X.columns, 'coefficient' : simplified_tree_clf.feature_importances_})
feature_importance

,feature,coefficient
0,count_before,0.352034
1,regularity,0.113566
2,period,0.531863
3,campaign_name_0_입회비반액할인,0.001395
4,campaign_name_1_입회비무료,0.000783
5,class_name_0_종일,0.000360
6,class_name_1_주간,0.000000
7,gender_F,0.000000


>> 새로운 데이터 예측

In [26]:
count_before = 3
regularity = 1
period = 10
campaign_name = '입회비무료'
class_name = '종일'
gender = 'M'

In [27]:
if campaign_name == '입회비반액할인' :
  campaign_name_list = [1, 0]
elif campaign_name == '입회비무료' :
  campaign_name_list = [0, 1]
elif campaign_name == '일반' :
  campaign_name_list = [0, 0]

if class_name == '종일' :
  class_name_list = [1, 0]
elif class_name == '주간' :
  class_name_list = [0, 1]
elif class_name == '야간' :
  class_name_list = [0, 0]

if gender == 'F' :
  gender_list = [1]
elif gender == 'M' :
  gender_list = [0]

input_data = [count_before, regularity, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [33]:
import numpy as np

input_data = np.array(input_data)
input_data.shape
input_data.reshape(1, -1)

array([[ 3,  1, 10,  0,  1,  1,  0,  0]])

카테고리 변수를 if문으로 분기하며 더미 변수로 작성합니다. 예측은 1이나 0뿐만 아니라 확률로 나타낼 수 있습니다.

In [34]:
print(simplified_tree_clf.predict(input_data))
print(simplified_tree_clf.predict_proba(input_data))

C:\Users\LGCARE\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


ValueError: Expected 2D array, got 1D array instead:
array=[ 3.  1. 10.  0.  1.  1.  0.  0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.